In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# NPBのプロセス数
npb_process :list[int] = [2, 4, 8, 16, 32, 64, 128, 256]
train_npb_process :list[int] = npb_process[:-1]
test_npb_process :list[int] = npb_process[-1:]
# NPBのCGの初期変数
cg_na: list[int] = [14000, 30000, 75000, 100000, 1500000]
cg_nonzer: list[int] = [11, 12, 13, 14, 15, 18, 21]
cg_niter: list[int] = [15, 30, 75, 90, 100]
cg_shift: list[int] = [20, 40, 60, 80, 110, 200]

train_cg_na: list[int] = cg_na[:-1]
train_cg_nonzer: list[int] = cg_nonzer[:-1]
train_cg_niter: list[int] = cg_niter[:-1]
train_cg_shift: list[int] = cg_shift[:-1]

test_cg_na: list[int] = cg_na[-1:]
test_cg_nonzer: list[int] = cg_nonzer[-1:]
test_cg_niter: list[int] = cg_niter[-1:]
test_cg_shift: list[int] = cg_shift[-1:]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]

# Extra-Pのオプション
modelerNames: list[str] = [
    # "refining", 
    "multi-parameter",
    "default", 
    # "basic --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1"
    ]

modelerOption: str = """ --options \#spm=Basic \#spo=poly_exponents=-1,0,1,2,3,log_exponents=0,1,force_combination_exponents=1,allow_negative_exponents=True"""

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]
list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

/workspace


DEBUG:__main__:hello
DEBUG:lib.lab_lib:hello


# TODO

* `Exclusive` の予測
    * Extra-P 単体でのモデル構築
    * Extra-P と Extra-P の組み合わせによるモデル構築
    * Extra-P 単体での予測
    * Extra-P と Extra-p の組み合わせによる予測
* `Inclusive` の予測
    * Extra-P 単体でのモデル構築
    * Extra-P と Extra-P の組み合わせによるモデル構築
    * Extra-P 単体での予測
    * Extra-P と Extra-p の組み合わせによる予測
* `#Call` の予測
    * Extra-P 単体でのモデル構築
    * Extra-P と Extra-P の組み合わせによるモデル構築
    * Extra-P 単体での予測
* `#Subrs` の予測
    * Extra-P 単体でのモデル構築
    * Extra-P と Extra-P の組み合わせによるモデル構築
    * Extra-P 単体での予測
    * Extra-P と Extra-p の組み合わせによる予測



In [2]:
# 関数コール回数予測のためのモデル

resVar = "#Call"
expVar: list[str] = ["process", "iteration", "size"]


dict_symbols_lulesh = {}
for elem in expVar:
    dict_symbols_lulesh[elem] = symbols(elem, real=True)

target_env_lulesh = [
    (dict_symbols_lulesh["size"], test_lulesh_sizes[-1]),
    (dict_symbols_lulesh["iteration"], test_lulesh_iterations[-1]),
    (dict_symbols_lulesh["process"], test_lulesh_processes[-1]),
]

modelerName: str = "multi-parameter"
modelerOption: str = """ --options \#spm=Basic \#spo=poly_exponents=-1,0,1,2,3,log_exponents=0,1,force_combination_exponents=1,allow_negative_exponents=True"""

print(f"resVar = {resVar}")

trainDF_lulesh: pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process=train_lulesh_processes,
    list_iteration=train_lulesh_iterations,
    list_size=train_lulesh_sizes,
    list_csvDir=list_csvDir,
    resVar=resVar,
)

functionNames_lulesh: list[str] = sorted(list(set(trainDF_lulesh["Name"])))

dict_functionName_model_call: dict[str, str] = {}

for functionName in functionNames_lulesh:
    trainDF_perFunc: pd.DataFrame = trainDF_lulesh[
        trainDF_lulesh["Name"] == functionName
    ].reset_index()

    model_fromExtraP_all = get_ExtraP_model(
        inputDF_perFunc=trainDF_perFunc,
        expVar=expVar,
        resVar=resVar,
        functionName=functionName,
        benchmarkName="lulesh",
        modelerName=modelerName,
        modelerOption=modelerOption,
        dict_symbols=dict_symbols_lulesh,
    )
    dict_functionName_model_call[functionName] = model_fromExtraP_all

resVar = #Call


In [3]:
list_resVar: list[str] = ["Exclusive", "Inclusive", "#Subrs"]
dict_resVar_functionName_models_all :dict = {}
dict_resVar_functionName_models_perCall :dict = {}

for resVar in list_resVar:
    pass

    print(f"resVar = {resVar}")

    trainDF_lulesh: pd.DataFrame = ret_averaged_rawDF_lulesh(
        list_process=train_lulesh_processes,
        list_iteration=train_lulesh_iterations,
        list_size=train_lulesh_sizes,
        list_csvDir=list_csvDir,
        resVar=resVar,
    )

    print(f"{trainDF_lulesh.head()}")

    functionNames_lulesh: list[str] = sorted(list(set(trainDF_lulesh["Name"])))

    dict_functionName_model_all: dict[str, str] = {}
    dict_functionName_model_perCall: dict[str, str] = {}

    for functionName in functionNames_lulesh:
        trainDF_perFunc: pd.DataFrame = trainDF_lulesh[
            trainDF_lulesh["Name"] == functionName
        ].reset_index()

        model_fromExtraP_all = get_ExtraP_model(
            inputDF_perFunc=trainDF_perFunc,
            expVar=expVar,
            resVar=resVar,
            functionName=functionName,
            benchmarkName="lulesh",
            modelerName=modelerName,
            modelerOption=modelerOption,
            dict_symbols=dict_symbols_lulesh,
        )
        dict_functionName_model_all[functionName] = model_fromExtraP_all

        model_fromExtraP_perCall = get_ExtraP_model(
            inputDF_perFunc=trainDF_perFunc,
            expVar=expVar,
            resVar=f"{resVar}PerCall",
            functionName=functionName,
            benchmarkName="lulesh",
            modelerName=modelerName,
            modelerOption=modelerOption,
            dict_symbols=dict_symbols_lulesh,
        )
        dict_functionName_model_perCall[functionName] = model_fromExtraP_perCall

    dict_resVar_functionName_models_all[resVar] = dict_functionName_model_all
    dict_resVar_functionName_models_perCall[resVar] = dict_functionName_model_perCall

resVar = Exclusive
   %Time  Exclusive Inclusive  #Call     #Subrs  \
0  100.0   0.000003     4,647    1.0      1.000   
1  100.0   0.025000     4,647    1.0    100.125   
2   84.0   6.192000     3,901    1.0      0.000   
3   15.1   0.727667       699    1.0      0.000   
4    0.2   0.006000        11    8.0  32768.000   

                                 Name  process  iteration  size  \
0                    .TAU_application        8          8    16   
1               int_main(int_char_**)        8          8    16   
2                      MPI_Finalize()        8          8    16   
3                          MPI_Init()        8          8    16   
4  void_CalcKinematicsForElems(Domain        8          8    16   

   ExclusivePerCall  
0          0.000003  
1          0.025000  
2          3.901000  
3          0.699000  
4          0.000750  
resVar = Inclusive
   %Time Exclusive  Inclusive  #Call     #Subrs  \
0  100.0    0.0035   6.966333    1.0      1.000   
1  100.0        25

In [4]:
dict_resultDF: dict[str, pd.DataFrame] = {}
for resVar in list_resVar:

    testDF_lulesh = ret_averaged_rawDF_lulesh(
        list_process=test_lulesh_processes,
        list_iteration=test_lulesh_iterations,
        list_size=test_lulesh_sizes,
        list_csvDir=list_csvDir,
        resVar=resVar,
    )

    print(f"{testDF_lulesh.head()}")

    functionNames_lulesh: list[str] = sorted(list(set(testDF_lulesh["Name"])))
    _list_series: list[pd.Series] = []

    for functionName in functionNames_lulesh:
        tmp_series: pd.Series = pd.Series(dtype="object")

        testDF_perFunc: pd.DataFrame = testDF_lulesh[
            (testDF_lulesh["Name"] == functionName)
            & (testDF_lulesh["process"] == test_lulesh_processes[-1])
            & (testDF_lulesh["iteration"] == test_lulesh_iterations[-1])
            & (testDF_lulesh["size"] == test_lulesh_sizes[-1])
        ].reset_index()

        if len(testDF_perFunc) == 1:
            tmp_series[f"real_{resVar}"] = testDF_perFunc.loc[0][resVar]
            tmp_series["functionName"] = functionName
            tmp_series[f"predicted_{resVar}"] = (
                dict_resVar_functionName_models_all[resVar][functionName]
                .subs(target_env_lulesh)
                .evalf()
            )

            # コール回数予測
            _predicted_call: float
            try:
                _predicted_call = (
                    dict_functionName_model_call[functionName]
                    .subs(target_env_lulesh)
                    .evalf()
                )
            except:
                _predicted_call = dict_functionName_model_call[functionName]

            # 予測対象値の１コール当たりの値
            _target_num_perCall: float = (
                dict_resVar_functionName_models_perCall[resVar][functionName]
                .subs(target_env_lulesh)
                .evalf()
            )

            # 予測されたコール回数 * 予測対象値の1コールあたりの値
            _predicted_num: float = _predicted_call * _target_num_perCall

            tmp_series[f"predicted_{resVar}_from_call"] = _predicted_num
            tmp_series["predicted_call"] = _predicted_call
            tmp_series["real_call"] = testDF_perFunc.loc[0][resVar]
            _list_series.append(tmp_series)

    print(resVar)
    _DF_tmp :pd.DataFrame = pd.DataFrame(data=_list_series)
    _DF_tmp = add_relativeErrorRateCol(
        inputDF=_DF_tmp,
        real_colName=f"real_{resVar}",
        predicted_colName=f"predicted_{resVar}",
        targetColName=f"real vs predicted all @{resVar}",)
    _DF_tmp = add_relativeErrorRateCol(
        inputDF=_DF_tmp,
        real_colName=f"real_{resVar}",
        predicted_colName=f"predicted_{resVar}_from_call",
        targetColName=f"real vs predicted using call @{resVar}",)
    dict_resultDF[resVar] = _DF_tmp

   %Time  Exclusive Inclusive  #Call      #Subrs  \
0  100.0   0.000004  2:13.310    1.0         1.0   
1  100.0  73.073333  2:13.310    1.0      2828.0   
2   17.9  14.006667    23,890  256.0  67108900.0   
3   14.6  18.807333    19,437  255.0         0.0   
4    7.7   3.580667    10,238  769.0     10503.8   

                                 Name  process  iteration  size  \
0                    .TAU_application      512        256    64   
1               int_main(int_char_**)      512        256    64   
2  void_CalcKinematicsForElems(Domain      512        256    64   
3                     MPI_Allreduce()      512        256    64   
4                void_CommSend(Domain      512        256    64   

   ExclusivePerCall  
0          0.000004  
1         72.675000  
2          0.054781  
3          0.076224  
4          0.000563  
Exclusive
   %Time Exclusive   Inclusive  #Call      #Subrs  \
0  100.0   0.00374  132.158333    1.0         1.0   
1  100.0  1:12.675  132.158333    1.

In [5]:
list_resVar: list[str] = ["Exclusive", "Inclusive", "#Subrs"]

print(dict_resultDF.keys())

print(dict_resultDF[list_resVar[0]].mean())

dict_resultDF[list_resVar[0]]

dict_keys(['Exclusive', 'Inclusive', '#Subrs'])
real_Exclusive                             1.453734e+02
predicted_Exclusive                        1.682026e+02
predicted_Exclusive_from_call              1.005981e+04
predicted_call                             6.352180e+07
real_call                                  1.453734e+02
real vs predicted all @Exclusive           9.842000e+01
real vs predicted using call @Exclusive    6.690583e+05
dtype: float64


/tmp/ipykernel_21744/3090816874.py:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(dict_resultDF[list_resVar[0]].mean())


,real_Exclusive,functionName,predicted_Exclusive,predicted_Exclusive_from_call,predicted_call,real_call,real vs predicted all @Exclusive,real vs predicted using call @Exclusive
0,3.366667e-06,.TAU_application,3.58708333333333e-6,2.10570898355123e-6,0.999999999999999,3.366667e-06,6.54702970297031,37.4541886073892
1,5.133010e+02,MPI_Allreduce(),437.915990349888,12124.7559208733,69114.4138888889,5.133010e+02,14.6863165374920,2262.11422165031
2,3.833333e-02,MPI_Barrier(),0.000816330555555555,18.5004563418933,662.485549774719,3.833333e-02,97.8704420289855,48162.0600223303
3,9.000000e-03,MPI_Comm_rank(),0.0165749444775296,0.00575717034518276,7682.31799535170,9.000000e-03,84.1660497503293,36.0314406090804
4,4.553333e-07,MPI_Comm_size(),4.85952941689757e-7,4.90953764193764e-7,0.988953189305555,4.553333e-07,6.72465776495400,7.82293503523359
5,7.433333e-01,MPI_Finalize(),0.488460112539233,130774.528268482,32774.5344861112,7.433333e-01,34.2878772368745,17592885.8657150
6,6.153333e-01,MPI_Init(),0.493019444444445,15072.7188036146,30426.8863055556,6.153333e-01,19.8776634163958,2449420.93233174
7,5.633333e-02,MPI_Irecv(),0.0607189920652085,-0.0896068042327797,-37413.2240107461,5.633333e-02,7.78519301516302,259.065332957597
8,8.950000e-01,MPI_Isend(),3.17498669266321,1.11110221315045,26027.4814559232,8.950000e-01,254.747116498683,24.1454986760275
9,7.166667e-02,MPI_Reduce(),0.0581820601722947,786.657959823452,16456.6172194788,7.166667e-02,18.8157299921470,1097562.26952110


In [6]:
print(dict_resultDF[list_resVar[1]].mean())
dict_resultDF[list_resVar[1]]

real_Inclusive                             3.516647e+02
predicted_Inclusive                        3.853701e+02
predicted_Inclusive_from_call              1.046825e+04
predicted_call                             6.352180e+07
real_call                                  3.516647e+02
real vs predicted all @Inclusive           1.170237e+02
real vs predicted using call @Inclusive    6.688036e+05
dtype: float64


/tmp/ipykernel_21744/261304090.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(dict_resultDF[list_resVar[1]].mean())


,real_Inclusive,functionName,predicted_Inclusive,predicted_Inclusive_from_call,predicted_call,real_call,real vs predicted all @Inclusive,real vs predicted using call @Inclusive
0,4.651960e+03,.TAU_application,4112.13002651492,3755.07819771439,0.999999999999999,4.651960e+03,11.6043617773413,19.2796600003743
1,5.133010e+02,MPI_Allreduce(),439.345968776950,12124.7559208733,69114.4138888889,5.133010e+02,14.4077317642182,2262.11422165031
2,3.833333e-02,MPI_Barrier(),0.000869176944444444,18.5004563418933,662.485549774719,3.833333e-02,97.7325818840580,48162.0600223303
3,9.000000e-03,MPI_Comm_rank(),0.0165754414998621,0.00575717034518276,7682.31799535170,9.000000e-03,84.1715722206905,36.0314406090804
4,4.553333e-07,MPI_Comm_size(),4.85952941689757e-7,4.90953764193764e-7,0.988953189305555,4.553333e-07,6.72465776495400,7.82293503523359
5,7.433333e-01,MPI_Finalize(),0.501003301920031,130774.528268482,32774.5344861112,7.433333e-01,32.6004526564981,17592885.8657150
6,6.153333e-01,MPI_Init(),0.503405555555556,15072.7188036146,30426.8863055556,6.153333e-01,18.1897797038642,2449420.93233174
7,8.066667e-02,MPI_Irecv(),0.0635597911803491,-0.0896068042327797,-37413.2240107461,8.066667e-02,21.2068704375837,211.082815164603
8,8.950000e-01,MPI_Isend(),0.426386241911113,1.11110221315045,26027.4814559232,8.950000e-01,52.3590791160767,24.1454986760275
9,7.166667e-02,MPI_Reduce(),0.0600932981444429,786.657959823452,16456.6172194788,7.166667e-02,16.1488863100797,1097562.26952110


In [7]:
print(dict_resultDF[list_resVar[2]].mean())
dict_resultDF[list_resVar[2]]

real_#Subrs                             6.717856e+07+0.000000e+00j
predicted_#Subrs                        6.091424e+07+0.000000e+00j
predicted_#Subrs_from_call              1.192909e+08+0.000000e+00j
predicted_call                          6.352180e+07+0.000000e+00j
real_call                               6.717856e+07+0.000000e+00j
real vs predicted all @#Subrs          N000000000000a000000000000N
real vs predicted using call @#Subrs   N000000000000a000000000000N
dtype: complex128


/tmp/ipykernel_21744/1563136227.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(dict_resultDF[list_resVar[2]].mean())


,real_#Subrs,functionName,predicted_#Subrs,predicted_#Subrs_from_call,predicted_call,real_call,real vs predicted all @#Subrs,real vs predicted using call @#Subrs
0,1.000000e+00,.TAU_application,0.999999999999999,0.999999999999998,0.999999999999999,1.000000e+00,1.11022302462516e-13,2.22044604925031e-13
1,0.000000e+00,MPI_Allreduce(),1012881.74466418,0,69114.4138888889,0.000000e+00,zoo,nan
2,0.000000e+00,MPI_Barrier(),-12.4034790238095,0,662.485549774719,0.000000e+00,zoo,nan
3,0.000000e+00,MPI_Comm_rank(),0.00833333333333333,0,7682.31799535170,0.000000e+00,zoo,nan
4,0.000000e+00,MPI_Comm_size(),0,0,0.988953189305555,0.000000e+00,nan,nan
5,0.000000e+00,MPI_Finalize(),10781.0178333333,0,32774.5344861112,0.000000e+00,zoo,nan
6,0.000000e+00,MPI_Init(),10802.6457222222,0,30426.8863055556,0.000000e+00,zoo,nan
7,1.360193e+04,MPI_Irecv(),609.931260790102,0,-37413.2240107461,1.360193e+04,95.5158487705907,100.000000000000
8,0.000000e+00,MPI_Isend(),27108.9717494639,0,26027.4814559232,0.000000e+00,zoo,nan
9,0.000000e+00,MPI_Reduce(),57.6759273283490,0,16456.6172194788,0.000000e+00,zoo,nan


In [8]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# NPBのプロセス数
npb_process :list[int] = [2, 4, 8, 16, 32, 64, 128, 256]
train_npb_process :list[int] = npb_process[:-1]
test_npb_process :list[int] = npb_process[-1:]
# NPBのCGの初期変数
cg_na: list[int] = [14000, 30000, 75000, 100000, 1500000]
cg_nonzer: list[int] = [11, 12, 13, 14, 15, 18, 21]
cg_niter: list[int] = [15, 30, 75, 90, 100]
cg_shift: list[int] = [20, 40, 60, 80, 110, 200]

train_cg_na: list[int] = cg_na[:-1]
train_cg_nonzer: list[int] = cg_nonzer[:-1]
train_cg_niter: list[int] = cg_niter[:-1]
train_cg_shift: list[int] = cg_shift[:-1]

test_cg_na: list[int] = cg_na[-1:]
test_cg_nonzer: list[int] = cg_nonzer[-1:]
test_cg_niter: list[int] = cg_niter[-1:]
test_cg_shift: list[int] = cg_shift[-1:]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]

# Extra-Pのオプション
modelerNames: list[str] = [
    # "refining", 
    "multi-parameter",
    "default", 
    # "basic --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1"
    ]

modelerOption: str = """ --options \#spm=Basic \#spo=poly_exponents=-1,0,1,2,3,log_exponents=0,1,force_combination_exponents=1,allow_negative_exponents=True"""

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]
list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

DEBUG:__main__:hello


In [9]:
# benchmark_lulesh.html と同様の方法を再現する

from types import MethodDescriptorType
from unittest import result


expVar :list[str] = ["process", "iteration", "size"]
resVar :str = "Exclusive"

trainDF_lulesh :pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process=train_lulesh_processes,
    list_iteration=train_lulesh_iterations,
    list_size=train_lulesh_sizes,
    list_csvDir=list_csvDir,
    resVar = resVar
)

testDF_lulesh :pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process = test_lulesh_processes[-1:],
    list_iteration=test_lulesh_iterations[-1:],
    list_size=test_lulesh_sizes[-1:],
    list_csvDir=list_csvDir,
    resVar = resVar
)

trainDF_lulesh = trainDF_lulesh.reset_index()
testDF_lulesh = testDF_lulesh.reset_index()

functionNames_lulesh :list[str] = sorted(list(set(trainDF_lulesh["Name"])))

dict_symbols_lulesh = {}
for elem in expVar:
    dict_symbols_lulesh[elem] = symbols(elem, real=True)

target_env = [
    (dict_symbols_lulesh["size"], test_lulesh_sizes[-1]),
    (dict_symbols_lulesh["iteration"], test_lulesh_iterations[-1]),
    (dict_symbols_lulesh["process"], test_lulesh_processes[-1])
]

modelerName :str = "multi-parameter"

resVar_all :str = "Exclusive"
resVar_perCall :str = "ExclusivePerCall"
resVar_call :str = "#Call"

dict_models_lulesh_all = {}
dict_models_lulesh_perCall = {}
dict_models_lulesh_call = {}

# モデル構築
for functionName in functionNames_lulesh:

    trainDF_lulesh_perFunc :pd.DataFrame = trainDF_lulesh[trainDF_lulesh["Name"] == functionName]
    testDF_lulesh_perFUnc :pd.DataFrame = testDF_lulesh[testDF_lulesh["Name"] == functionName]

    str_ExtraP_model_all :str = get_ExtraP_model(
        inputDF_perFunc = trainDF_lulesh_perFunc,
        expVar = expVar,
        resVar = resVar_all,
        functionName = functionName,
        benchmarkName = "lulesh",
        modelerName = modelerName,
        modelerOption = modelerOption,
        dict_symbols=dict_symbols_lulesh,
    )
    dict_models_lulesh_all[functionName] = str_ExtraP_model_all

    str_ExtraP_model_perCall :str = get_ExtraP_model(
        inputDF_perFunc = trainDF_lulesh_perFunc,
        expVar = expVar,
        resVar = resVar_perCall,
        functionName = functionName,
        benchmarkName = "lulesh",
        modelerName = modelerName,
        modelerOption=modelerOption,
        dict_symbols=dict_symbols_lulesh,
    )
    dict_models_lulesh_perCall[functionName] = str_ExtraP_model_perCall

    str_ExtraP_model_call :str = get_ExtraP_model(
        inputDF_perFunc = trainDF_lulesh_perFunc,
        expVar = expVar,
        resVar = resVar_call,
        functionName = functionName,
        benchmarkName="lulesh",
        modelerName=modelerName,
        modelerOption=modelerOption,
        dict_symbols=dict_symbols_lulesh,
    )
    dict_models_lulesh_call[functionName] = str_ExtraP_model_call

_list_series :list[pd.Series] = []
# 予測
for functionName in functionNames_lulesh:
    # 全体
    _all :float = dict_models_lulesh_all[functionName].subs(target_env_lulesh).evalf()
    # １コール当たり
    _perCall :float = dict_models_lulesh_perCall[functionName].subs(target_env_lulesh).evalf()
    # コール回数
    _call :float = dict_models_lulesh_call[functionName].subs(target_env_lulesh).evalf()

    target_lulesh_DF :pd.DataFrame = testDF_lulesh[
        (testDF_lulesh["Name"] == functionName) &
        (testDF_lulesh["process"] == test_lulesh_processes[-1]) &
        (testDF_lulesh["iteration"] == test_lulesh_iterations[-1]) &
        (testDF_lulesh["size"] == test_lulesh_sizes[-1])
        ]
    target_lulesh_DF = target_lulesh_DF.reset_index()
    # 実測値
    _real_exclusive :float = target_lulesh_DF.loc[0][resVar_all]
    _real_call :float = target_lulesh_DF.loc[0][resVar_call]

    _tmp_series :pd.Series = pd.Series(dtype="object")
    _tmp_series["all to all"] = _all
    _tmp_series["perCall to all"] = _perCall * _call
    _tmp_series["real exclusive"] = _real_exclusive
    _tmp_series["real call"] = _real_call

    _list_series.append(_tmp_series)

resultDF :pd.DataFrame = pd.DataFrame(data=_list_series)
resultDF = add_relativeErrorRateCol(inputDF=resultDF, real_colName="real exclusive", predicted_colName="all to all", targetColName="vs all to all")
resultDF = add_relativeErrorRateCol(inputDF = resultDF, real_colName="real exclusive", predicted_colName="perCall to all", targetColName="vs perCall to all")

print(resultDF.mean())

resultDF

# "vs all to all" より "vs perCall to all" のほうが精度がMAPEベースで良いことを確認

all to all           1.682026e+02
perCall to all       1.061613e+02
real exclusive       1.453734e+02
real call            6.717865e+07
vs all to all        9.842000e+01
vs perCall to all    7.618677e+01
dtype: float64


,all to all,perCall to all,real exclusive,real call,vs all to all,vs perCall to all
0,3.58708333333333e-6,2.10570898355123e-6,3.366667e-06,1.000000e+00,6.54702970297031,37.4541886073892
1,437.915990349888,179.465101548773,5.133010e+02,1.023000e+03,14.6863165374920,65.0370637211358
2,0.000816330555555555,0.0279258262284882,3.833333e-02,1.000000e+00,97.8704420289855,27.1500185343787
3,0.0165749444775296,0.00691026689926807,9.000000e-03,9.221000e+03,84.1660497503293,23.2192566747993
4,4.85952941689757e-7,4.96437818799606e-7,4.553333e-07,1.000000e+00,6.72465776495400,9.02733941426189
5,0.488460112539233,3.99012618543521,7.433333e-01,1.000000e+00,34.2878772368745,436.788276067517
6,0.493019444444445,0.495375000000000,6.153333e-01,1.000000e+00,19.8776634163958,19.4948537378115
7,0.0607189920652085,0.0641030960977833,5.633333e-02,3.773280e+04,7.78519301516302,13.7924782800888
8,3.17498669266321,1.14257579197692,8.950000e-01,3.773280e+04,254.747116498683,27.6620996622256
9,0.0581820601722947,0.0478019236476088,7.166667e-02,1.000000e+00,18.8157299921470,33.2996414219412
